In [160]:
import numpy as np
import matplotlib.pyplot as plt
import pandas
from numpy.linalg import inv
import random
import csv
import math


cov = []
mean0 = []
mean1 = []
with open('./data/DS1_Cov.txt') as f:
    for line in f:
        to_add = line.split(',')
        to_add = [float(i) for i in to_add[0:len(to_add) - 1]]
        cov.append(to_add)

with open('./data/DS1_m_0.txt') as f:
    for line in f:
        to_add = line.split(',')
        to_add = [float(i) for i in to_add[0:len(to_add) - 1]]
        mean0.append(to_add)

        
with open('./data/DS1_m_1.txt') as f:
    for line in f:
        to_add = line.split(',')
        to_add = [float(i) for i in to_add[0:len(to_add) - 1]]
        mean1.append(to_add)
    

In [161]:
positive = np.random.multivariate_normal(mean1[0],cov, 2000)
negative = np.random.multivariate_normal(mean0[0],cov, 2000)

In [162]:
def sampling(data_array):
    np.random.shuffle(data_array)
    test_set = []
    train_set = []
    for row in data_array[0:600]:
        test_set.append(row)
    
    for row in data_array[600:2001]:
        train_set.append(row)
        
    return test_set,train_set
    

In [163]:
test1,train1 = sampling(positive)
t_1 = [1 for i in range(0,1400)]
test2,train2 = sampling(negative)
t_0 = [0 for i in range(0,1400)]
X=list(train1)
#t = t_1
#t.extend(t_2)
X.extend(train2)
def MLE_estimator(train1, train2, t_0,t_1):
    MLE_mean0 = (1/len(t_0))*(np.matmul(np.transpose(train1),t_0))
    MLE_mean1 = (1/len(t_1))*(np.matmul(np.transpose(train2),t_1))
    pi = len(t_1)/(len(t_1) + len(t_0))
    S_1 = 1/(len(t_1)) * (np.matmul(np.transpose([(i - mean1)[0] for i in train1]),([(i - mean1)[0] for i in train1])))
    #S_1 = positive
    S_2 = 1/(len(t_1)) * (np.matmul(np.transpose([(i - mean0)[0] for i in train2]), [(i - mean0)[0] for i in train2]))

    sigma = (len(t_1)/(len(t_1) + len(t_0)))*S_1 + (len(t_0)/(len(t_1) + len(t_0)))*S_2
    
    omega = np.matmul(inv(sigma), np.transpose(np.subtract(mean1, mean0)))
    omega0 = (-1/2)*np.matmul(np.matmul(mean1,inv(sigma)),np.transpose(mean1)) + (1/2)*np.matmul(np.matmul(mean0,inv(sigma)),np.transpose(mean0))
    return omega0,omega

q2_omega0,q2_omega = MLE_estimator(train1,train2, t_0, t_1)


If entries are negative then, we know that probability of it being positive is less than $\frac{1}{2}$, if entries are positive, the probabilitiy is greater than $\frac{1}{2}$. Hence we can select for positive and negative entries.

In [164]:

pred1 = np.matmul(test1,q2_omega) + q2_omega0
pred2 = np.matmul(test2,q2_omega) + q2_omega0


def count(mat):
    positive = 0
    negative = 0
    for i in mat:
        if (i > 0):
            positive +=1
        else:
            negative +=1
    return positive, negative
print(count(pred1))
print(count(pred2))

accuracy = (count(pred1)[0] + count(pred2)[1])/1200
precision = (count(pred1)[0])/(count(pred1)[0] + count(pred1)[1])
recall = (count(pred1)[0])/(count(pred1)[0] + count(pred2)[0])



print("The accuracy of this classifier is " + str(100*accuracy) + "%")
print("The precision of this classifier is " + str(100*precision) + "%")
print("The recall of this classifier is " + str(100*recall) + "%")
print("The F-measure of this classifier is " + str(2*precision/(precision + recall)))


(575, 25)
(27, 573)
The accuracy of this classifier is 95.66666666666667%
The precision of this classifier is 95.83333333333334%
The recall of this classifier is 95.51495016611295%
The F-measure of this classifier is 1.0016638935108153


In [165]:
def distance(vector1, vector2):
    out = 0
    for i in range(0,len(vector1)):
        out += pow((vector1[i] - vector2[i]),2)
    return math.sqrt(out)

def knearestneighbourhood(train,point,k):
    distances = []
    length = len(train)
    for x in range(0,length):
        dist = distance(point,X[x])
        if (x < length/2):
            distances.append((dist,1))
        else:
            distances.append((dist,0))
    distances = sorted(distances)[:k]
    frequency_p = 0
    frequency_n = 0 
    
    for dist in distances:
        if dist[1] == 0:
            frequency_n += 1
        elif dist[1] == 1:
            frequency_p += 1
    return 0 if frequency_n>frequency_p else 1


In [166]:
def resultsknearest(test,train,k):
    out = []
    for i in test:
        out.append(knearestneighbourhood(train,i,k))
    return out



In [167]:
positive_values = []
negative_values = []
for k in range(1,13):
    sample1 = resultsknearest(test1, train1,k)
    sample2 = resultsknearest(test2, train2,k)
    positive_values.append(count(sample1))
    negative_values.append(count(sample2))

In [177]:
print(positive_values)
print(negative_values)
def best_fit(positive_values,negative_values):
    
    highest_f_measure = -1
    index = 0
    for i in range(0,len(positive_values)):
        accuracy = (positive_values[i][0] + negative_values[i][1])/1200
        precision = (positive_values[i][0])/(positive_values[i][0] + positive_values[i][1])
        recall = (positive_values[i][0])/(positive_values[i][0] + negative_values[i][0])
        F_measure = 2*precision/(precision + recall)
        if (F_measure > highest_f_measure):
            highest_f_measure = F_measure
            index = i
    accuracy = (positive_values[index][0] + negative_values[index][1])/1200
    precision = (positive_values[index][0])/(positive_values[index][0] + positive_values[index][1])
    recall = (positive_values[index][0])/(positive_values[index][0] + negative_values[index][0])
    F_measure = 2*precision/(precision + recall)
    
    return accuracy,precision,recall,F_measure,index

accuracy,precision,recall,F_measure,index = best_fit(positive_values,negative_values)
print("value of k that performs the best is ", index)

print("The accuracy of this classifier is " + str(100*accuracy) + "%")
print("The precision of this classifier is " + str(100*precision) + "%")
print("The recall of this classifier is " + str(100*recall) + "%")
print("The F-measure of this classifier is " + str(2*precision/(precision + recall)))
    



    



[(312, 288), (462, 138), (321, 279), (416, 184), (325, 275), (409, 191), (319, 281), (396, 204), (314, 286), (388, 212), (323, 277), (386, 214)]
[(315, 285), (475, 125), (297, 303), (422, 178), (308, 292), (398, 202), (294, 306), (392, 208), (306, 294), (375, 225), (310, 290), (370, 230)]
value of k that performs the best is  1
The accuracy of this classifier is 48.91666666666667%
The precision of this classifier is 77.0%
The recall of this classifier is 49.30629669156883%
The F-measure of this classifier is 1.2192582953806117


In [169]:
q4_cov1 = []
q4_cov2 = []
q4_cov3 = []
q4_c1_mean1 = []
q4_c1_mean2 = []
q4_c1_mean3 = []
q4_c2_mean1 = []
q4_c2_mean2 = []
q4_c2_mean3 = []
probability = [0.1,0.42,0.48]

with open('./data/DS2_Cov1.txt') as f:
    for line in f:
        to_add = line.split(',')
        to_add = [float(i) for i in to_add[0:len(to_add) - 1]]
        q4_cov1.append(to_add)
with open('./data/DS2_Cov2.txt') as f:
    for line in f:
        to_add = line.split(',')
        to_add = [float(i) for i in to_add[0:len(to_add) - 1]]
        q4_cov2.append(to_add)
with open('./data/DS2_Cov3.txt') as f:
    for line in f:
        to_add = line.split(',')
        to_add = [float(i) for i in to_add[0:len(to_add) - 1]]
        q4_cov3.append(to_add)      
with open('./data/DS2_c1_m1.txt') as f:
    for line in f:
        to_add = line.split(',')
        to_add = [float(i) for i in to_add[0:len(to_add) - 1]]
        q4_c1_mean1.append(to_add)
with open('./data/DS2_c1_m2.txt') as f:
    for line in f:
        to_add = line.split(',')
        to_add = [float(i) for i in to_add[0:len(to_add) - 1]]
        q4_c1_mean2.append(to_add)
with open('./data/DS2_c1_m3.txt') as f:
    for line in f:
        to_add = line.split(',')
        to_add = [float(i) for i in to_add[0:len(to_add) - 1]]
        q4_c1_mean3.append(to_add)
with open('./data/DS2_c2_m1.txt') as f:
    for line in f:
        to_add = line.split(',')
        to_add = [float(i) for i in to_add[0:len(to_add) - 1]]
        q4_c2_mean1.append(to_add)
with open('./data/DS2_c2_m2.txt') as f:
    for line in f:
        to_add = line.split(',')
        to_add = [float(i) for i in to_add[0:len(to_add) - 1]]
        q4_c2_mean2.append(to_add)        
with open('./data/DS2_c2_m3.txt') as f:
    for line in f:
        to_add = line.split(',')
        to_add = [float(i) for i in to_add[0:len(to_add) - 1]]
        q4_c2_mean3.append(to_add)        
        
sampling1 = np.random.multinomial(2000,probability)
sampling2 = np.random.multinomial(2000,probability)
class1_m1 = np.random.multivariate_normal(q4_c1_mean1[0],q4_cov1, sampling1[0])
class1_m2 = np.random.multivariate_normal(q4_c1_mean2[0],q4_cov2, sampling1[1])
class1_m3 = np.random.multivariate_normal(q4_c1_mean3[0],q4_cov3, sampling1[2])
class2_m1 = np.random.multivariate_normal(q4_c2_mean1[0],q4_cov1, sampling1[0])
class2_m2 = np.random.multivariate_normal(q4_c2_mean2[0],q4_cov2, sampling1[1])
class2_m3 = np.random.multivariate_normal(q4_c2_mean3[0],q4_cov3, sampling1[2])
positive2 = list(class1_m1)
positive2.extend(list(class1_m2))
positive2.extend(list(class1_m3))
negative2 = list(class2_m1)
negative2.extend(list(class2_m2))
negative2.extend(list(class2_m3))

q5_test1, q5_train1 = sampling(positive2)
q5_t1 = [1 for i in range(0,1400)]
q5_test2, q5_train2 = sampling(negative2)
q5_t2 = [0 for i in range(0,1400)]

q5_omega0, q5_omega = MLE_estimator(q5_train1,q5_train2,q5_t1, q5_t2)

In [170]:
q5_pred1 = np.matmul(q5_test1,q5_omega) + q5_omega0
q5_pred2 = np.matmul(q5_test2,q5_omega) + q5_omega0


In [173]:
print(count(q5_pred1))
print(count(q5_pred2))

(209, 391)
(222, 378)


In [174]:
q5_positive_values = []
q5_negative_values = []
for k in range(1,13):
    sample1 = resultsknearest(q5_test1, q5_train1,k)
    sample2 = resultsknearest(q5_test2, q5_train2,k)
    q5_positive_values.append(count(sample1))
    q5_negative_values.append(count(sample2))

In [175]:
print(q5_positive_values)
print(q5_negative_values)

[(298, 302), (444, 156), (287, 313), (403, 197), (306, 294), (391, 209), (299, 301), (384, 216), (299, 301), (370, 230), (310, 290), (372, 228)]
[(302, 298), (454, 146), (292, 308), (424, 176), (295, 305), (405, 195), (309, 291), (409, 191), (319, 281), (392, 208), (318, 282), (385, 215)]


In [178]:
q5_accuracy,q5_precision,q5_recall,q5_F_measure,q5_index = best_fit(q5_positive_values,q5_negative_values)
print("value of k that performs the best is ", q5_index)
print("The accuracy of this classifier is " + str(100*q5_accuracy) + "%")
print("The precision of this classifier is " + str(100*q5_precision) + "%")
print("The recall of this classifier is " + str(100*q5_recall) + "%")
print("The F-measure of this classifier is " + str(2*q5_precision/(q5_precision + q5_recall)))


value of k that performs the best is  1
The accuracy of this classifier is 49.166666666666664%
The precision of this classifier is 74.0%
The recall of this classifier is 49.44320712694877%
The F-measure of this classifier is 1.1989319092122832
